In [ ]:
%%configure -f
{"name": "gerard-ztf-query", "executorMemory": "12G", "numExecutors": 16, "executorCores": 4,
 "conf": {"spark.yarn.appMasterEnv.PYSPARK_PYTHON":"python3"}}

In [ ]:
import matplotlib.pyplot as plt
import subprocess

In [ ]:
def list_hdfs(dir):
    cmd = '/opt/hadoop/bin/hdfs dfs -ls {}'.format(dir)
    return subprocess.check_output(
        cmd,
        stderr = subprocess.STDOUT,
        shell=True).decode('utf-8')

In [ ]:
import time
hdfs_dir = 'hdfs:///ztf'
print(list_hdfs(hdfs_dir))

In [ ]:
t = time.time()
ztf = spark.read.parquet('hdfs:///ztf/full_subset.parquet')#hdfs_dir)# .cache(), data size is too large for cluster to cache
ztf.createOrReplaceTempView('ztf')
print(time.time()-t)

In [ ]:
t = time.time()
spark.sql('SELECT count(*) FROM ztf').show()
print(time.time()-t)

In [ ]:
t = time.time()
spark.sql('SELECT count(*) FROM ztf').show()
print(time.time()-t)

In [ ]:
t = time.time()
df = spark.sql('SELECT * FROM ztf where cardinality(mag_r)>30 limit 10 ')#.show()
print(time.time()-t)
df.show()

In [ ]:
ls=df.select('mjd_r','mag_r').toPandas()

In [ ]:
fig, ax = plt.subplots(1,1)
for i,row in ls.iterrows():
    x=row['mjd_r']
    y=row['mag_r']
#     a=ax.plot(x,y,"-o");
    y=[i for _,i in sorted(zip(x,y))]
    a=ax.plot(sorted(x),y,"-o");
    break
%matplot plt

In [ ]:
t = time.time()
spark.sql('SELECT ps1_objid, count(*) as num FROM ztf GROUP BY ps1_objid order by num desc, ps1_objid limit 100').show()
print(time.time()-t)

In [ ]:
t = time.time()
df=spark.sql('SELECT mjd_r, mag_r FROM ztf where array_min(mag_r)< 17 limit 10')
print(time.time()-t)

In [ ]:
t = time.time()
spark.sql('SELECT mjd_r, mag_r FROM ztf WHERE ps1_objid=132002123692096071').show()
print(time.time()-t)

In [ ]:
t = time.time()
df = spark.sql('SELECT cardinality(mag_r) as card, count(*) as num FROM ztf group by card ')#.show()
print(time.time()-t)
df.show()